bkln vs stsg
questions:
1. the relationship between bkln and stsg
2. do bkln fill missing stsg on map

or
1. Map bkln to stsg


In [1]:
import geopandas as gp
import pandas as pd
stsg_path = 'dcdata\Street_Segments.geojson'
bkln_path = 'dcdata\Bike_Lane_Street_RightofWay.geojson'

Populating the interactive namespace from numpy and matplotlib


In [2]:
gp_stsg = gp.read_file(stsg_path)
gp_bkln = gp.read_file(bkln_path)
gp_stsg.shape, gp_bkln.shape

((13522, 22), (1310, 19))

In [3]:
# gp_bkln[gp_bkln.TRAVELDIRECTION=='In Bound']
gp_bkln.TRAVELDIRECTION.value_counts()

             964
Out Bound    181
In Bound     165
dtype: int64

In [5]:
gp_bkln = gp_bkln[['OBJECTID','STREETSEGID','geometry','TRAVELDIRECTION']]
gp_stsg = gp_stsg[['OBJECTID','STREETSEGID','geometry']]

# method 2 for question 1: area of buffered linestring

- buffer lines by 10 meter
- get area of intersection after buffered
- calculate the percentage of area overlap, both of bkln and stsg, return the max percentage

In [11]:
gp_bkln_bfr = gp_bkln.copy()
gp_bkln_bfr.geometry = gp_bkln_bfr.buffer(0.000098)
gp_stsg_bfr = gp_stsg.copy()
gp_stsg_bfr.geometry = gp_stsg_bfr.buffer(0.000098)
bk_st_bfr_sjoin=gp.tools.sjoin(gp_bkln_bfr,gp_stsg_bfr,how='left')
bk_st_bfr_sjoin = bk_st_bfr_sjoin.rename(columns={'geometry':'geometry_left'})

In [13]:
bk_st_bfr_intrsetn = pd.merge(
    bk_st_bfr_sjoin[['OBJECTID_left','STREETSEGID_left','geometry_left','index_right']],
    gp_stsg_bfr[['STREETSEGID','geometry','OBJECTID']],
    left_on='index_right',right_index = True)
print bk_st_bfr_intrsetn.shape

(8501, 7)


(1310L,)

In [15]:
bk_st_bfr_intrsetn['intersection']=bk_st_bfr_intrsetn.apply(lambda x: x.geometry_left.intersection(x.geometry),axis=1)
bk_st_bfr_intrsetn['area'] = bk_st_bfr_intrsetn.intersection.apply(lambda x: x.area)

In [16]:
def get_area_percentage(x):
    area_left = x.geometry_left.area
    area_right = x.geometry.area
    return x.area/area_left if area_left<area_right else x.area/area_right
bk_st_bfr_intrsetn['area_prcnt'] = bk_st_bfr_intrsetn.apply(get_area_percentage, axis=1)

## 5 bklns that do not exist in stsg

In [157]:
objid_bk_group = {}
# 14857, 15395,  15422
group_name = {'nbk':'new bike lanes', 
              'mbkN1st':'segid matched bike lane N : 1 street segment', 
              '_mbkN1st':'segid not match bike lane N : 1 street segment',
              'm_m':'segid match/not match bk  N : 1 street segment',
              'mbk11st':'segid matched bike lane 1 : 1 street segment',
              '_mbk11st':'segid not match bike lane 1 : 1 street segment',
              '_mbk1Nst':'segid not match bike lane 1 : N street segment',
             }


In [158]:
bk_st_ovlp = bk_st_bfr_intrsetn[bk_st_bfr_intrsetn.area_prcnt>0.9]
objid_new_bkln = set(gp_bkln.OBJECTID.values) - set(bk_st_ovlp.OBJECTID_left.values)
objid_new_bkln
objid_bk_group['nbk']=set(objid_new_bkln)

In [159]:
# bk_st_bfr_intrsetn[bk_st_bfr_intrsetn.area_prcnt>0.9].to_csv('bk_st_brf.csv')
print bk_st_ovlp[bk_st_ovlp.STREETSEGID==bk_st_ovlp.STREETSEGID_left].shape
segid_match_bk = bk_st_ovlp[bk_st_ovlp.STREETSEGID==bk_st_ovlp.STREETSEGID_left]
a = segid_match_bk.STREETSEGID_left.value_counts()
segids_bkln_share = a[a>1].index.tolist()
segids_bkln_match = a[a==1].index.tolist()
objid_bk_group['mbkN1st'] = set(segid_match_bk[segid_match_bk.STREETSEGID_left.isin(segids_bkln_share)].OBJECTID_left.tolist())
objid_bk_group['mbk11st'] = set(segid_match_bk[segid_match_bk.STREETSEGID_left.isin(segids_bkln_match)].OBJECTID_left.tolist())
print len(objid_bk_group['mbk11st'])



(1292, 10)
1282


In [160]:
segid_not_match_bk = bk_st_ovlp[bk_st_ovlp.STREETSEGID!=bk_st_ovlp.STREETSEGID_left]

b = segid_not_match_bk.OBJECTID_left.value_counts()
objid_bk_group['_mbk1Nst'] = set(b[b>1].index.tolist())

c = segid_not_match_bk.STREETSEGID_left.value_counts()

segid_bkln_repeat = c[c>1].index.tolist()
bkln_N1_stsg = segid_not_match_bk[segid_not_match_bk.STREETSEGID_left.isin(segid_bkln_repeat) & ~segid_not_match_bk.OBJECTID_left.isin(objid_bkln_multi_stsg)]
objid_bk_group['_mbkN1st'] = set(bkln_N1_stsg.OBJECTID_left.tolist())

objid_bk_group['_mbk11st'] = set(segid_not_match_bk[segid_not_match_bk.STREETSEGID_left.isin(c[c==1].index)].OBJECTID_left.tolist())

objid_bk_group['m_m'] = objid_bk_group['mbk11st'] & objid_bk_group['_mbk11st']

objid_bk_group['mbk11st'] = objid_bk_group['mbk11st'] - objid_bk_group['m_m']
objid_bk_group['_mbk11st'] = objid_bk_group['_mbk11st'] - objid_bk_group['m_m']


In [161]:
s = 0
for k, v in group_name.items():
    print v, len(objid_bk_group[k])
    s+=len(objid_bk_group[k])
print s

segid matched bike lane 1 : 1 street segment 1281
segid match/not match bk  N : 1 street segment 1
segid not match bike lane 1 : N street segment 4
segid not match bike lane N : 1 street segment 4
new bike lanes 5
segid matched bike lane N : 1 street segment 10
segid not match bike lane 1 : 1 street segment 5
1310


In [169]:
stsg_clr = '#DDEBF7'
gp_stsg['color'] = stsg_clr

group_color = {'nbk':'#FF0000', 
              'mbkN1st':'#FFFF00', 
              '_mbkN1st':'#FFC000',
              'm_m':'#92D050',
              'mbk11st':'#00B050',
              '_mbk11st':'#7030A0',
              '_mbk1Nst':'#0070C0',
            }

In [170]:
with open('C:\Users\JHW\Desktop\stsg_bkln.js','w') as f:
    for key, objids in objid_bk_group.items():
        gp_df = gp_bkln[gp_bkln.OBJECTID.isin(objids)].copy()
        gp_df['color']=group_color[key]
        js = gp_df.to_json()
        f.write('var %s = %s;\n' % (key, js))
    js_stsg = gp_stsg.to_json()
    f.write('var stsg = %s;\n' % js_stsg)
#     js_bkln = gp_bkln.to_json()
#     f.write('var bkln = %s;\n' % js_bkln)

# Question 2
see whether these matches have same STREETSEGID
- a few bkln overlap with multiple stsg
- 


In [14]:
print bkln_stsg_intersection.shape
print bkln_stsg_intersection[bkln_stsg_intersection.not_point].shape
bkln_stsg_intersection[bkln_stsg_intersection.not_point].groupby('STREETSEGID_left')['STREETSEGID'].apply(list)
bkln_stsg_intersection[bkln_stsg_intersection.not_point].

(6788, 9)
(1263, 9)


STREETSEGID_left
0                   [6647]
3                      [3]
8                      [8]
10                    [10]
12                    [12]
27                    [27]
29                    [29]
32                    [32]
48                    [48]
57                    [57]
62                    [62]
77                    [77]
79                    [79]
88                    [88]
97                    [97]
99                    [99]
104                  [104]
132                  [132]
139                  [139]
142                  [142]
153                  [153]
173                  [173]
184                  [184]
189                  [189]
195                  [195]
241                  [241]
243                  [243]
247                  [247]
252                  [252]
264                  [264]
                 ...      
13062              [13062]
13069              [13069]
13070              [13070]
13085              [13085]
13091              [13091]
13109      

In [49]:
b[b.STREETSEGID==b.STREETSEGID_left]

NameError: name 'b' is not defined

In [73]:
b[(b.STREETSEGID==b.STREETSEGID_left)& b.not_point]

,OBJECTID_left,STREETSEGID_left,geometry_left,index_right,STREETSEGID,geometry,OBJECTID,intersection,in_type,not_point
0,1,6816,LINESTRING (-77.02191316958731 38.893884722960...,248,6816,LINESTRING (-77.02191316958731 38.893884722960...,249,(LINESTRING (-77.02191316958731 38.89388472296...,<class 'shapely.geometry.multilinestring.Multi...,True
1,2,11841,LINESTRING (-77.02191007827204 38.894802308188...,249,11841,LINESTRING (-77.02191007899236 38.894802307625...,250,"(POINT (-77.021910023266 38.89480618179751), L...",<class 'shapely.geometry.collection.GeometryCo...,True
3,4,3324,LINESTRING (-77.02191815762689 38.897283446264...,251,3324,LINESTRING (-77.02191815762689 38.897283446264...,252,"(POINT (-77.02191815762689 38.89728344626466),...",<class 'shapely.geometry.collection.GeometryCo...,True
175,176,3935,LINESTRING (-77.01990289522601 38.897347327262...,3382,3935,LINESTRING (-77.01990289522601 38.897347327262...,3383,"(POINT (-77.02191815762689 38.89728344626466),...",<class 'shapely.geometry.collection.GeometryCo...,True
2,3,8839,LINESTRING (-77.02191953817406 38.896139598618...,250,8839,LINESTRING (-77.02191953889447 38.896139598618...,251,"(POINT (-77.02191972309645 38.89626047977487),...",<class 'shapely.geometry.multipoint.MultiPoint'>,True
770,771,11188,LINESTRING (-77.02191667720315 38.898454118349...,2752,11188,LINESTRING (-77.02191667720315 38.898454118349...,2753,"(POINT (-77.02191667720315 38.89845411834963),...",<class 'shapely.geometry.collection.GeometryCo...,True
780,781,2783,LINESTRING (-77.02191667720315 38.898454118349...,252,2783,LINESTRING (-77.02191667720315 38.898454118349...,253,"(POINT (-77.02191667720315 38.89845411834963),...",<class 'shapely.geometry.collection.GeometryCo...,True
4,5,12836,LINESTRING (-77.02191672467082 38.899820921064...,253,12836,LINESTRING (-77.02191672467082 38.899820921064...,254,"(POINT (-77.02191672467082 38.89982092106483),...",<class 'shapely.geometry.collection.GeometryCo...,True
5,6,12493,LINESTRING (-77.02191327176027 38.900877908623...,254,12493,LINESTRING (-77.02191327176027 38.900877908623...,255,(LINESTRING (-77.02191327176027 38.90087790862...,<class 'shapely.geometry.multilinestring.Multi...,True
6,7,7336,LINESTRING (-77.02192130916271 38.907247644956...,260,7336,LINESTRING (-77.02192130988324 38.907247644955...,261,LINESTRING (-77.02191709895898 38.908455824521...,<class 'shapely.geometry.linestring.LineString'>,True


In [80]:
b[~(b.STREETSEGID==b.STREETSEGID_left)& b.not_point].OBJECTID_left.value_counts()

281     3
859     2
535     2
962     2
1274    1
1175    1
565     1
274     1
1297    1
336     1
1165    1
684     1
199     1
1221    1
dtype: int64

In [ ]:
281     3
859     2
535     2
962     2
1274    2
1175    2
274     2
336     2